In [77]:
# Gestion des imports
import configparser
import requests, json
import pandas as pd
import re
from collections import Counter

In [78]:
# IMPORTS GOOGLE
import googleplaces
from googleplaces import GooglePlaces, types, lang
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import RegexpTokenizer  
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from gensim import models, corpora
from nltk.tokenize import sent_tokenize, word_tokenize, WordPunctTokenizer
from nltk.tokenize import RegexpTokenizer  
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from gensim import models, corpora
from nltk.tokenize import sent_tokenize, word_tokenize, WordPunctTokenizer

In [79]:
# Gestion de la configuration
config = configparser.ConfigParser()
config.read('../Configuration.conf')

['../Configuration.conf']

In [80]:
# Obtention de ma Google API KEY
mon_api_key = config.get('Google', 'api_key')

In [81]:
# Fonction de request simple sur Google
def RequeteSimpleGoogle(query):
    # enter your api key here 
    api_key = mon_api_key
    # Création d'un dataframe de récolte des réponses
    Reponses_df = pd.DataFrame()
    # url variable store url
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
    # get method of requests module --> return response object 
    r = requests.get(url + 'query=' + query + '&key=' + api_key) 
    # json method of response object convert json format data into python format data 
    x = r.json()
    # now x contains list of nested dictionaries
    # store the value of result key in variable y 
    y = x['results']
    #print ('------------------------------------------------- Résultat en JSON : \n', y, '\n')
    # keep looping upto lenght of y 
    for i in range(len(y)):
        # Print value corresponding to the 'name' key at the ith index of y
        try:
            df = pd.DataFrame({'ADRESSE': y[i]['formatted_address'],
                               'LATITUDE': y[i]['geometry']['location']['lat'],
                               'LONGITUDE': y[i]['geometry']['location']['lng'],
                               'NOM': y[i]['name']}, index = [i])
            Reponses_df = Reponses_df.append(df)
        except: pass
    return Reponses_df

In [82]:
# La recherche à faire sur Google
query = input('Recherche sur Google Maps : ')
reponses = RequeteSimpleGoogle(query)
reponses

Recherche sur Google Maps : genesis lyon


,ADRESSE,LATITUDE,LONGITUDE,NOM
0,"50 Quai Paul Sédallian, 69009 Lyon, France",45.788695,4.814934,Genesis Groupe


In [83]:
# Quel index du dataframe est à chercher
index = input('Index du dataframe à chercher : ')
MonAdresse = reponses.ADRESSE[int(index)]
MaLatitude = reponses.LATITUDE[int(index)]
MaLongitude = reponses.LONGITUDE[int(index)]
MonNom = reponses.NOM[int(index)]

Index du dataframe à chercher : 0


In [84]:
# CHARGEMENT DE LA LIBRAIRIE STOPWORDS DE NLTK
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/Fred/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [85]:
# CHARGEMENT DE LA LIBRAIRIE PUNKT DE NLTK
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/Fred/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [86]:
# CHARGEMENT DES STOPWORDS NLTK FRANCAIS
stopwords = nltk.corpus.stopwords.words('french')

In [87]:
# CHARGEMENT DES STEMMERS NLTK (SnowballStemmer) FRANCAIS
stemmer = SnowballStemmer("french")

In [88]:
##### GOOGLE PLACES
# Recherche des urls de sites web des entreprises en fonction de la localisation via Google Places
# Processus:
# Se connecter à gmail, puis accéder "cloud platform console" suivant la description
# https://support.google.com/cloud/answer/6158862
# Créer un projet et lui associer une clé API
# Cette clé sera utilisée dans tout lors de la génération des urls de sites webs des entrepises.
# Ne pas oublier d'activer les services API de "Google Places" via API et services > Bibliothèques > Places API
# Source : https://console.cloud.google.com/apis/library/places-backend.googleapis.com
# Exécuter le code de géoloc et renseigner les urls dans un dataframe
####
# Export des urls à partir des paramètres de google place seach API. 
# Description des paramètres: https://developers.google.com/places/web-service/search#PlaceSearchRequests 
####
# AVEC fredchezgenesis@gmail.com
# CREATION SUR Google Cloud Platform DU PROJET 42351
# API_KEY = c'est secret !

In [89]:
# DATAFRAMES POUR SORTIES
WebAccueil_df = pd.DataFrame({'NOM': 'NOM',
                        'PAGE WEB' : 'PAGE WEB',
                        'TOTAL MOTS PAGE' : 'TOTAL MOTS PAGE',    
                        'MOT' : 'MOT',
                        'ITERATIONS' : 'ITERATIONS',
                        'RANG PAGE' : 'RANG PAGE'},
                      index = [0])
WebSite_df = pd.DataFrame({'NOM': 'NOM',
                        'PAGE WEB' : 'PAGE WEB',
                        'TOTAL MOTS PAGE' : 'TOTAL MOTS PAGE',    
                        'MOT' : 'MOT',
                        'ITERATIONS' : 'ITERATIONS',
                        'RANG PAGE' : 'RANG PAGE'},
                      index = [0])

In [90]:
# FONCTION D'ASPIRATION DE PAGES WEB
def AspirationPages(MesPages):
    AspirationPages = ''
    for i in range (0, len(MesPages)):
        response = requests.get(MesPages[i])
        soupCurrent = BeautifulSoup(response.text, "html.parser")
        for paragraph in soupCurrent.find_all('p'):
            AspirationPages = AspirationPages + ' ' + paragraph.text
            
    # ELIMINATION DE CERTAINS CARACTERES
    AspirationPages = AspirationPages.replace('/', ' ')
    AspirationPages = AspirationPages.replace("'", ' ')
    # TOKENIZE (TOKEN = JETON)
    # Premier tokenize par phrase puis par mot pour s'assurer que la ponctuation a son propre jeton
    tokens = [word.lower() for envoye in nltk.sent_tokenize(AspirationPages) for word in nltk.word_tokenize(envoye)]
    # Filtrage des jetons ne contenant aucune lettre (numérique, ponctuation)
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    # Filtrage des stopwords
    filtered_tokens_SW = [w for w in filtered_tokens if not w in stopwords]
    TopFinal = Counter(filtered_tokens_SW)
    return (TopFinal.most_common(),len(filtered_tokens_SW))

In [91]:
# Extraction des urls d'entreprise en fonction des coordonnées géographiques à travers l'API Google Map
# Clé API
google_places = GooglePlaces(mon_api_key)

query_result = google_places.nearby_search( language = 'fr', keyword = None, 
                            location = MonAdresse, 
                            lat_lng = {'lat': MaLatitude, 'lng': MaLongitude}, 
                            name = MonNom, radius = 3200, 
                            rankby = 'prominence', sensor = False, type = None,
                            types = [], pagetoken = None)

for place in query_result.places:
    place.get_details()
    if place.website is not None:
            
        # AFFICHAGE DES INFOS GOOGLE PLACE
        print('******** google_places.nearby_search ********')
        print('** name :', place.name)
        print('** website :', place.website)
        print('** formatted_address :', place.formatted_address)
        print('** geo_location :', place.geo_location)
        print('** local_phone_number :', place.local_phone_number)
        print('** types :', place.types)
        print('** url :', place.url)
            
        # ANALYSE DES DIFFERENTS PAGES POUR CHAQUE SITE CIBLE
        response = requests.get(place.website)
        soup = BeautifulSoup(response.text, "html.parser")
        # AFFICHAGE DES INFOS BEAUTIFULSOUP
        print('******** BeautifulSoup ********')
        url_list = []
        url_list.append(place.website)
        url_list_Temp = []
        for url in soup.find_all('a'):
                url_list.append(url.get('href')) 
        # NETTOYAGE DES PAGES TROUVEES DANS LA LISTE DES HREF
        url_list = [w for w in url_list if w is not None]
        url_list_Temp = [place.website + w for w in url_list if w.endswith('.html')]
        url_list.extend(url_list_Temp)
        url_list = [w for w in url_list if w.startswith(place.website)]
        # EFFACEMENT DES DOUBLONS POTENTIELS DE LA LISTE
        url_list = list(set(url_list))
        # AFFICHAGE
        print('** Nb pages trouvées sur le site ', place.website, ' :', len(url_list))
        [print(w) for w in url_list]

******** google_places.nearby_search ********
** name : Genesis Groupe
** website : https://genesis-groupe.fr/
** formatted_address : 50 Quai Paul Sédallian, 69009 Lyon, France
** geo_location : {'lat': Decimal('45.7886948'), 'lng': Decimal('4.8149336')}
** local_phone_number : 04 72 86 05 05
** types : ['point_of_interest', 'establishment']
** url : https://maps.google.com/?cid=16557938292905174678
******** BeautifulSoup ********
** Nb pages trouvées sur le site  https://genesis-groupe.fr/  : 11
https://genesis-groupe.fr/a-propos-de-genesis-groupe/
https://genesis-groupe.fr/pole-ingenierie/
https://genesis-groupe.fr/big-data/
https://genesis-groupe.fr/pole-infra
https://genesis-groupe.fr/
https://genesis-groupe.fr/pole-conseil/
https://genesis-groupe.fr/genesis-groupe-une-ssii-qui-recrute-de-facon-recurrente/
https://genesis-groupe.fr/recherche-et-developpement/
https://genesis-groupe.fr/author/stephane-r/
https://genesis-groupe.fr/contact/
https://genesis-groupe.fr/pole-infra/


In [92]:
# ANALYSE DES MOTS DE LA PAGE D ACCUEIL
url_accueil = []
url_accueil.append(place.website)
AnalyseAccueil, NbMotsAccueil = AspirationPages(url_accueil)
print('******** Mots page accueil ******** Nombre de mots utiles : ', NbMotsAccueil)
print(AnalyseAccueil)

# ANALYSE DES MOTS DE TOUTES LES PAGES 
AnalyseToutLeSite, NbMotsSite = AspirationPages(url_list)
print('******** Mots de tout le site ******** Nombre de mots utiles : ', NbMotsSite)
print(AnalyseToutLeSite)

******** Mots page accueil ******** Nombre de mots utiles :  125
[('lire', 6), ('suite', 6), ('les', 5), ('genesis', 4), ('groupe', 4), ('développement', 3), ('projets', 2), ('digitale', 2), ('administration', 2), ('données', 2), ('entreprises', 2), ('esn', 2), ('entreprise', 2), ('services', 2), ('découvrez', 2), ('approche', 1), ('processus', 1), ('cadrage', 1), ('expression', 1), ('besoins', 1), ('pilotage', 1), ('programmes', 1), ('relations', 1), ('moe-moa', 1), ('conception', 1), ('tests', 1), ('recette', 1), ('support', 1), ('équipes', 1), ('innovation', 1), ('r', 1), ('agilité', 1), ('design', 1), ('user', 1), ('digital', 1), ('expérience', 1), ('reactive', 1), ('programming…', 1), ('pôle', 1), ('tecno-addict', 1), ('agile', 1), ('service', 1), ('transformation', 1), ('clients', 1), ('futurs', 1), ('partenaires', 1), ('architecture', 1), ('ingénierie', 1), ('systèmes', 1), ('réseaux', 1), ('sécurité', 1), ('intégration', 1), ('application', 1), ('bases', 1), ('devops…', 1), ('m